## DKL vs GPFormer Comparison
This notebook converts the provided script into an executable form in Codex.


Python >= 3.9, PyTorch >= 1.12, GPyTorch >= 1.9 설치
matplotlib, scipy, scikit-learn 필요
실행 후 각 모델 결과:
콘솔 출력: MAE, σ vs MAE 상관도
저장 이미지: DKL_mae_sigma.png, GPFormer_mae_sigma.png


In [12]:
# prompt: python 버전 확인

!python --version
!!pip3 install torch torchvision
!pip install gpytorch

Python 3.11.13


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gpytorch
from gpytorch.models import ApproximateGP
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.kernels import ScaleKernel, RBFKernel
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import numpy as np
from urllib.request import urlretrieve
import pandas as pd
from pathlib import Path
from sklearn.datasets import make_regression


In [14]:
def load_kin8nm():
    url = 'https://raw.githubusercontent.com/yaringal/DropoutUncertaintyExps/master/uci_datasets/kin8nm/kin8nm.csv'
    cache = Path('kin8nm.csv')
    try:
        if not cache.exists():
            urlretrieve(url, cache)
        df = pd.read_csv(cache)
        X = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
        y = torch.tensor(df.iloc[:, -1].values, dtype=torch.float32)
    except Exception as e:
        # fallback synthetic data
        X_np, y_np = make_regression(n_samples=2000, n_features=8, noise=0.1, random_state=0)
        X = torch.tensor(X_np, dtype=torch.float32)
        y = torch.tensor(y_np, dtype=torch.float32)
    return X, y


In [15]:
class DKLFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
    def forward(self, x):
        return self.net(x)

In [16]:
class DKLGPModel(ApproximateGP):
    def __init__(self, feature_extractor, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super().__init__(variational_strategy)
        self.feature_extractor = feature_extractor
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = ScaleKernel(RBFKernel())
    def forward(self, x):
        x_feat = self.feature_extractor(x)
        mean_x = self.mean_module(x_feat)
        covar_x = self.covar_module(x_feat)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [17]:
class DKLModelWrapper(nn.Module):
    def __init__(self, gp_model, likelihood):
        super().__init__()
        self.gp_model = gp_model
        self.likelihood = likelihood
    def forward(self, x):
        return self.gp_model(x)
    def predict(self, x):
        self.eval()
        self.likelihood.eval()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            dist = self.likelihood(self.gp_model(x))
        return dist.mean, dist.variance.sqrt()

In [18]:
class GPFormer(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.mean_head = nn.Linear(d_model, 1)
        self.log_var_head = nn.Linear(d_model, 1)
    def forward(self, x):
        x = self.embedding(x)
        x = self.encoder(x.unsqueeze(1)).squeeze(1)
        mean = self.mean_head(x).squeeze(-1)
        sigma = torch.exp(0.5 * self.log_var_head(x).squeeze(-1))
        return mean, sigma

In [19]:
def train_gpformer(model, x, y, num_epochs=1000, lr=0.001):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        mean, sigma = model(x)
        nll = 0.5 * torch.log(sigma ** 2 + 1e-6) + 0.5 * ((y - mean) ** 2) / (sigma ** 2 + 1e-6)
        loss = nll.mean()
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'[GPFormer] Epoch {epoch:3d}/{num_epochs}, Loss: {loss.item():.4f}')

In [20]:
def train_dkl(wrapper, likelihood, x, y, num_epochs=1000, lr=0.01):
    wrapper.train()
    likelihood.train()
    optimizer = torch.optim.Adam(wrapper.parameters(), lr=lr)
    mll = gpytorch.mlls.VariationalELBO(likelihood, wrapper.gp_model, num_data=y.size(0))
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = wrapper(x)
        loss = -mll(output, y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'[DKL] Epoch {epoch:3d}/{num_epochs}, Loss: {loss.item():.4f}')

In [21]:
def evaluate_model(name, pred_mean, pred_sigma, y_true):
    mae = mean_absolute_error(y_true.numpy(), pred_mean.detach().numpy())
    corr, _ = spearmanr(torch.abs(y_true - pred_mean).numpy(), pred_sigma.detach().numpy())
    print(f'{name} Results - MAE: {mae:.4f}, Spearman Corr(|err|, sigma): {corr:.4f}')
    plt.figure(figsize=(6,4))
    plt.scatter(pred_sigma.detach().numpy(), torch.abs(y_true - pred_mean).detach().numpy(), alpha=0.5)
    plt.xlabel('Predicted Sigma')
    plt.ylabel('Absolute Error')
    plt.title(f'{name}: Error vs Sigma')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_mae_sigma.png')
    plt.close()
    return mae, corr


In [ ]:
X, y = load_kin8nm()
N = X.size(0)
train_size = int(0.8 * N)
train_x, test_x = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]
print('Training DKL...')
feature_extractor = DKLFeatureExtractor(input_dim=X.shape[1])
inducing_points = train_x[:128]
gp_model = DKLGPModel(feature_extractor, inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()
dkl_model = DKLModelWrapper(gp_model, likelihood)
train_dkl(dkl_model, likelihood, train_x, train_y, num_epochs=30)
dkl_mean, dkl_sigma = dkl_model.predict(test_x)
evaluate_model('DKL', dkl_mean, dkl_sigma, test_y)
print('Training GPFormer...')
gpformer = GPFormer(input_dim=X.shape[1])
train_gpformer(gpformer, train_x, train_y, num_epochs=30)
gpformer.eval()
with torch.no_grad():
    mean_f, sigma_f = gpformer(test_x)
evaluate_model('GPFormer', mean_f, sigma_f, test_y)

Training DKL...
[DKL] Epoch   0/30, Loss: 11954.1592
[DKL] Epoch  10/30, Loss: 11092.5381
[DKL] Epoch  20/30, Loss: 10299.2959
DKL Results - MAE: 100.9842, Spearman Corr(|err|, sigma): 0.1338
Training GPFormer...
[GPFormer] Epoch   0/30, Loss: 5940.3062
